# Autores

In [16]:
%reload_ext cypher
import pandas as pd
pd.set_option('display.max_colwidth', -1)

## Top 10 autores que más escribieron

In [17]:
dfAutores = %%cypher MATCH p=(n:Author)-[r:WRITES]->({valid:True}) RETURN count(p) as Articles, n.name as Authors order by Articles DESC LIMIT 10
dfAutores = dfAutores.get_dataframe()
dfAutores.set_index('Authors')

10 rows affected.


,Articles
Authors,
LaSillaVacia.com,184
Jineth Prieto,111
Juan Esteban Lewin,109
Juanita León,94
Juanita Vélez,78
Laura Ardila Arrieta,67
Natalia Arenas,66
Antonio Canchila García,43
Ana Karietna León Quiroga,43


In [18]:
from nltk import ngrams
import operator
from itertools import islice
import string 
from tabulate import tabulate

def stopWordsInGrams(grams):    
    count = 0
    for w in grams:
        if w in stop_words_spanish:
            count = count + 1                
    return count/len(grams)  

def isNumber(s):
    try: 
        int(s)
        float(s)
        return True
    except ValueError:
        return False

def getNGram(text, n, stop_words, m):
    text = text.lower()
    ngramas = ngrams(text.split(), n)
    h_dict = {}   
    for grams in ngramas:
        if stopWordsInGrams(grams) > 0.34:
            continue
        words = ' '.join(grams)        
        words = words.lower() 
        words = ''.join(e for e in words if e.isalnum() or e == ' ')    
        words = words.strip()  
        if  words in stop_words or words in punct_signs or words == '' or words in abc or isNumber(words) or len(words.split(' ')) != n:
             continue
        if words in h_dict:
            h_dict[words] = h_dict[words] + 1
        else: 
            h_dict[words] = 1 
    sorted_dict = sorted(h_dict.items(), key=operator.itemgetter(1), reverse=True)
    if m is -1:
        return list(sorted_dict)
    else:
        return list(islice(sorted_dict, m))

In [19]:
stop_words_spanish = ['un', 'una', 'unas', 'unos', 'uno', 'sobre', 'todo', 'también', 'tras', 'otro', 'algún', 'alguno', 'alguna', 'algunos', 'algunas', 'ser', 'es', 'soy', 'eres', 'somos', 'sois', 'estoy', 'esta', 'estamos', 'estais', 'estan', 'como', 'en', 'para', 'atras', 'porque', 'por qué',  'estaba', 'ante', 'antes', 'siendo', 'ambos', 'pero', 'por', 'poder', 'puede', 'puedo', 'podemos', 'podeis', 'pueden', 'fui', 'fue', 'fuimos', 'fueron', 'hacer', 'hago', 'hace', 'hacemos', 'haceis', 'hacen', 'cada', 'fin', 'incluso', 'primero', 'desde', 'conseguir', 'consigo', 'consigue', 'consigues', 'conseguimos', 'consiguen', 'ir', 'voy', 'va', 'vamos', 'vais', 'van', 'vaya', 'gueno', 'ha', 'tener', 'tengo', 'tiene', 'tenemos', 'teneis', 'tienen', 'el', 'la', 'lo', 'las', 'los', 'su', 'aqui', 'mio', 'tuyo', 'ellos', 'ellas', 'nos', 'nosotros', 'vosotros', 'vosotras', 'si', 'dentro', 'solo', 'solamente', 'saber', 'sabes', 'sabe', 'sabemos', 'sabeis', 'saben', 'ultimo', 'largo', 'bastante', 'haces', 'muchos', 'aquellos', 'aquellas', 'sus', 'entonces', 'tiempo', 'verdad', 'verdadero', 'verdadera', 'cierto', 'ciertos', 'cierta', 'ciertas', 'intentar', 'intento', 'intenta', 'intentas', 'intentamos', 'intentais', 'intentan', 'dos', 'bajo', 'arriba', 'encima', 'usar', 'uso', 'usas', 'usa', 'usamos', 'usais', 'usan', 'emplear', 'empleo', 'empleas', 'emplean', 'ampleamos', 'empleais', 'valor', 'muy', 'era', 'eras', 'eramos', 'eran', 'modo', 'bien', 'cual', 'cuando', 'donde', 'mientras', 'quien', 'con', 'entre', 'sin', 'trabajo', 'trabajar', 'trabajas', 'trabaja', 'trabajamos', 'trabajais', 'trabajan', 'podria', 'podrias', 'podriamos', 'podrian', 'podriais', 'yo', 'aquel', 'a', 'en', 'de', 'para', 'por', 'según', 'sin', 'sobre', 'tras', 'bajo', 'contra', 'hacia', 'hasta', 'rt', 'que', 'se', 'no', 'del', 'al', 'http', 'https', 'y', 'más', 'todos', 'año', 'será', 'este', 'vía', 'les', 'ni', 'dice', 'le', 'está', 'qué', 'hay', 'htt', 'quiere', 'nuestra', 'sí', 'of', 'in', 'años', 'firma' , 'firman', 'nuevo', 'hoy', 'ayer', 'inicio', 'ya', 'día','son', 'final', 'and', 'with', 'you', 'mundo', 'gran', 'sol', 'recibe', 'millones',  'gracias', 'hermana', 'fuego', 'me', 'eso',  'we', 'after', 'camino', 'dijo', 'caminos', 'recibirá', 'entiendo', 'mejores', 'mejor', 'deseos', 'lleve', 'nuevamente', 'ahora', 'definitiva', 'ese', 'definitivo', 'han', 'sino', 'tal', 'mismos', 'estos', 'así', 'ver', 'esto', 'esta', 'sido', 'manera', 'sigo', 'da', 'esa', 'aunque', 'están', 'mil', 'además', 'había', 'él', 'después', 'pues', 'vez', 'quienes', 'mi']

abc = string.ascii_lowercase 
punct_signs = string.punctuation

## Unigramas y Bigramas frecuentes en los artículos de los 10 autores más populares

In [38]:
authors = dfAutores.reset_index()['Authors'].values
for a in authors:
    dfText = %%cypher MATCH (n:Author{name:{a}})-[:WRITES]->(p:Article) return p.text as Text
    dfText = dfText.get_dataframe()
    data_author = '\n'.join(dfText['Text'].values)
    print('\n-----------------', a,'-----------------\n')
    final = getNGram(data_author, 1, stop_words_spanish, 20)
    print(tabulate(final, ['Ngram', 'Number']), '\n')    
    final2 = getNGram(data_author, 2, stop_words_spanish, 20)
    print(tabulate(final2, ['Ngram', 'Number']))

184 rows affected.

----------------- LaSillaVacia.com -----------------

Ngram         Number
----------  --------
silla            520
farc             430
gobierno         368
paz              350
acuerdo          340
campaña          322
santos           270
parte            240
partido          224
presidente       218
tres             218
país             212
alcalde          210
armas            206
proceso          204
hecho            194
colombia         188
cambio           182
nacional         170
frente           166 

Ngram                       Number
------------------------  --------
san pablo                       92
cambio radical                  66
juan manuel                     48
manuel santos                   46
silla caribe                    42
partido liberal                 42
vargas lleras                   40
presidente santos               38
plan colombia                   36
partido conservador             34
centro democrático              34
acuerdo

Ngram                Number
-----------------  --------
cambio radical           71
silla caribe             64
vargas lleras            42
germán vargas            38
juan manuel              31
manuel santos            29
buena parte              27
álex char                23
presidente juan          23
opción ciudadana         21
kiko gómez               21
viejo pin                20
silla vacía              20
segunda vuelta           19
oneida pinto             19
control político         19
yahir acuña              19
o menos                  18
gobierno nacional        18
bandas criminales        18
66 rows affected.

----------------- Natalia Arenas -----------------

Ngram       Number
--------  --------
silla          361
gobierno       263
peñalosa       229
bogotá         229
paz            198
ciudad         177
ex             173
farc           165
alcalde        160
tres           147
hecho          135
partido        135
decisión       133
ciento         133
sólo     

## NGramas más frecuentes en la misma oración que 'Santos' en los 10 autores populares

In [39]:
def getNGramasFrecuencyBySentence(sentences, regExp, term, n, m, lower):
    regexpComp = re.compile(regExp)
    data_term = ''
    for s in sentences: 
        if regexpComp.search(s.lower() if lower == True else s): 
            data_term = data_term + '\n' + s

    stop_words = stop_words_spanish
    stop_words.append(term)
    finalTerm = getNGram(data_term, n,stop_words , m)
    print(tabulate(finalTerm, ['Ngram', 'Number']))

In [41]:
for a in authors:
    dfText = %%cypher MATCH (n:Author{name:{a}})-[:WRITES]->(p:Article) return p.text as Text
    dfText = dfText.get_dataframe()
    data_author = '\n'.join(dfText['Text'].values)
    print('\n-----------------', a,'-----------------\n')
    getNGramasFrecuencyBySentence(data_author.split('.'), r'[\n .,;\'\"]Santos[\n .,;\'\"]', 'santos', 1, 10, False)
    print('\n')
    getNGramasFrecuencyBySentence(data_author.split('.'), r'[\n .,;\'\"]Santos[\n .,;\'\"]', 'santos', 2, 10, False)

184 rows affected.

----------------- LaSillaVacia.com -----------------

Ngram         Number
----------  --------
presidente        84
juan              42
manuel            40
gobierno          36
paz               34
farc              28
campaña           24
durante           20
acuerdo           20
medios            18


Ngram                   Number
--------------------  --------
juan manuel                 40
presidente juan             26
manuel santos                8
plan colombia                6
unidad nacional              6
altos funcionarios           4
vicky dávila                 4
comisión nacional            4
arango bacci                 4
medios tradicionales         4
112 rows affected.

----------------- Jineth Prieto -----------------

Ngram          Number
-----------  --------
presidente         26
juan               20
manuel             19
santander          18
gobierno           15
silla              14
vuelta             13
segunda            11
presidenc

## NGramas más frecuentes en la misma oración que 'Uribe' en los 10 autores populares

In [42]:
for a in authors:
    dfText = %%cypher MATCH (n:Author{name:{a}})-[:WRITES]->(p:Article) return p.text as Text
    dfText = dfText.get_dataframe()
    data_author = '\n'.join(dfText['Text'].values)
    print('\n-----------------', a,'-----------------\n')
    getNGramasFrecuencyBySentence(data_author.split('.'), r'[\n .,;\'\"]Uribe[\n .,;\'\"]', 'uribe', 1, 10, False)
    print('\n')
    getNGramasFrecuencyBySentence(data_author.split('.'), r'[\n .,;\'\"]Uribe[\n .,;\'\"]', 'uribe', 2, 10, False)

184 rows affected.

----------------- LaSillaVacia.com -----------------

Ngram            Number
-------------  --------
gobierno             12
presidente           12
duque                12
álvaro               10
iván                  8
precandidatos         8
durante               8
zuluaga               8
partido               6
silla                 6


Ngram                           Number
----------------------------  --------
tres precandidatos                   6
carlos holmes                        4
iván zuluaga                         4
holmes trujillo                      4
precandidatos presidenciales         4
iván duque                           4
óscar iván                           4
centro democrático                   4
ex presidente                        4
duque aseguró                        2
112 rows affected.

----------------- Jineth Prieto -----------------

Ngram           Number
------------  --------
álvaro              16
presidente           8
ex   